## Tarea: Uso de RDDs y Dataframes en PySpark
### Datos Masivos I
#### *Manuel Alejandro León Rosas *

**Objetivo de la tarea:** explorar la carga, uso y manipulación de RDDs y DataFrames en PySpark.

**Instrucciones:**

1. Usando databricks, debes crear una libreta y cargar una base de datos (escoge la que prefieras de "FIFA 20 complete player dataset").
2. Debes cargar los datos a databricks y cargarlos a tu libreta como RDD y como dataFrame.
3. Una vez que cargues los datos, debes realizar dos operaciones usando el RDD y dos operaciones usando el DataFrame.
4. También debes generar una consulta en SQL.

In [0]:
#imports
import pyspark
from pyspark.sql.functions import * 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Primera_Tarea').getOrCreate()

####Carga de la base de datos

In [0]:
#RDD
datos_crudos18 = spark.sparkContext.textFile('/FileStore/tables/players_18.csv')

#DataFrame
df_fifa18 = spark.read.csv('/FileStore/tables/players_18.csv', header = True, inferSchema = True)

#### Operaciones con la BDD

**RDD**

In [0]:
nombres_columnas = datos_crudos18.take(1)[0]

for i, elem in enumerate(nombres_columnas.split(',')):
    print(f'Col_{i}: ', elem)

Col_0:  sofifa_id
Col_1:  player_url
Col_2:  short_name
Col_3:  long_name
Col_4:  age
Col_5:  dob
Col_6:  height_cm
Col_7:  weight_kg
Col_8:  nationality
Col_9:  club
Col_10:  overall
Col_11:  potential
Col_12:  value_eur
Col_13:  wage_eur
Col_14:  player_positions
Col_15:  preferred_foot
Col_16:  international_reputation
Col_17:  weak_foot
Col_18:  skill_moves
Col_19:  work_rate
Col_20:  body_type
Col_21:  real_face
Col_22:  release_clause_eur
Col_23:  player_tags
Col_24:  team_position
Col_25:  team_jersey_number
Col_26:  loaned_from
Col_27:  joined
Col_28:  contract_valid_until
Col_29:  nation_position
Col_30:  nation_jersey_number
Col_31:  pace
Col_32:  shooting
Col_33:  passing
Col_34:  dribbling
Col_35:  defending
Col_36:  physic
Col_37:  gk_diving
Col_38:  gk_handling
Col_39:  gk_kicking
Col_40:  gk_reflexes
Col_41:  gk_speed
Col_42:  gk_positioning
Col_43:  player_traits
Col_44:  attacking_crossing
Col_45:  attacking_finishing
Col_46:  attacking_heading_accuracy
Col_47:  attack

In [0]:
#Filtrado de jugadores mayores de 30 años

def filtrado_edades(renglon):
    renglon_separado = renglon.split(',')
    return (renglon_separado[4], 1)

In [0]:
edades = datos_crudos18.map(filtrado_edades)
first = edades.take(1)[0]
edades = edades.filter(lambda renglon : renglon != first )
edades = edades.map(lambda renglon : (int(renglon[0]), renglon[1]))
print(edades.take(5))
print(edades.count())

[(32, 1), (30, 1), (25, 1), (31, 1), (30, 1)]
17592


In [0]:
mayores = edades.filter(lambda renglon : renglon[0] > 30)
print(mayores.take(5))
print(mayores.count())

[(32, 1), (31, 1), (31, 1), (39, 1), (32, 1)]
2505


**DataFrame**

In [0]:
for i, elem in enumerate(df_fifa18.columns):
    print(f'Col_{i}: ', elem)

Col_0:  sofifa_id
Col_1:  player_url
Col_2:  short_name
Col_3:  long_name
Col_4:  age
Col_5:  dob
Col_6:  height_cm
Col_7:  weight_kg
Col_8:  nationality
Col_9:  club
Col_10:  overall
Col_11:  potential
Col_12:  value_eur
Col_13:  wage_eur
Col_14:  player_positions
Col_15:  preferred_foot
Col_16:  international_reputation
Col_17:  weak_foot
Col_18:  skill_moves
Col_19:  work_rate
Col_20:  body_type
Col_21:  real_face
Col_22:  release_clause_eur
Col_23:  player_tags
Col_24:  team_position
Col_25:  team_jersey_number
Col_26:  loaned_from
Col_27:  joined
Col_28:  contract_valid_until
Col_29:  nation_position
Col_30:  nation_jersey_number
Col_31:  pace
Col_32:  shooting
Col_33:  passing
Col_34:  dribbling
Col_35:  defending
Col_36:  physic
Col_37:  gk_diving
Col_38:  gk_handling
Col_39:  gk_kicking
Col_40:  gk_reflexes
Col_41:  gk_speed
Col_42:  gk_positioning
Col_43:  player_traits
Col_44:  attacking_crossing
Col_45:  attacking_finishing
Col_46:  attacking_heading_accuracy
Col_47:  attack

In [0]:
#Filtrado de jugadores mayores de 30 años (sin SQL)
df_fifa18small = df_fifa18.select('short_name', 'age')
df_fifa18small.show(5)

+-----------------+---+
|       short_name|age|
+-----------------+---+
|Cristiano Ronaldo| 32|
|         L. Messi| 30|
|           Neymar| 25|
|         M. Neuer| 31|
|        L. Suárez| 30|
+-----------------+---+
only showing top 5 rows



In [0]:
df_mayores = df_fifa18small.where(df_fifa18small.age > 30)
print(df_mayores.show(5))
print(df_mayores.count())

+-----------------+---+
|       short_name|age|
+-----------------+---+
|Cristiano Ronaldo| 32|
|         M. Neuer| 31|
|     Sergio Ramos| 31|
|        G. Buffon| 39|
|     G. Chiellini| 32|
+-----------------+---+
only showing top 5 rows

None
2505


####Consulta SQL

In [0]:
df_fifa18.registerTempTable('fifa18sql')

Utilicé también la consulta de los jugadores mayores de 30 años

In [0]:
consulta = 'select short_name as Nombre, age as Edad from fifa18sql where age > 30'
consulta2 = 'select count(*) as NumJugadoresMayores from fifa18sql where age > 30'


print(spark.sql(consulta).show(5))
print(spark.sql(consulta2).show())

+-----------------+----+
|           Nombre|Edad|
+-----------------+----+
|Cristiano Ronaldo|  32|
|         M. Neuer|  31|
|     Sergio Ramos|  31|
|        G. Buffon|  39|
|     G. Chiellini|  32|
+-----------------+----+
only showing top 5 rows

None
+-------------------+
|NumJugadoresMayores|
+-------------------+
|               2505|
+-------------------+

None
